In [1]:
import numpy as np
import math
import itertools
import picos as pic

In [3]:
def SDP(n,K,d,p,D):
    
    def Swap_multiplication(A,B):

        C = tuple(set(A)^set(B))

        return C 


    def K_fun(M,K,N):

        Fun=0

        for q in range(0,M+1):
            Fun+= math.comb(N-K,M-q)*math.comb(K,q)*(-1)**q
        return Fun 

    T=2**p ### Total number of reductions ###

    Generators=list(range(0, p))

    Elements=[]

    for i in range(n+1):
        Elements += (list(itertools.combinations(Generators, i)))


    Elements_R={new_label:old_label for old_label, new_label in enumerate(Elements)}

    rho=[]

    length=0

    for i in range(0,d):

            length+=math.comb(p,i)
            
    length_c=0

    for i in range(d,n+1):

            length_c+=math.comb(p,i)

    Rep=[]

    for i in range(T):
        for j in range(T):
            C=Swap_multiplication(Elements[i],Elements[j])
            Rep.append([C,i,j])  

    Rep=sorted(Rep)


    ##### SDP #####


    pic.available_solvers()

    F = pic.RealVariable('F', T) # the moment matrix
    
    Ap = pic.RealVariable("Ap", T)
    
    sdp = pic.Problem()

    sdp.set_objective('find') 


    ##### Adding state constraints #####

    sdp.add_constraint(F[0]==K**4)
    sdp.add_constraint(F[T-1]==K**3)
    
  
    ##### Adding K-L constrains #####
    
    
    
    """
    ### K*B'_j-A'_j=0  for d > j >= 0 ###
    
    for j in range(d):
        
        S=tuple(range(0,j))
        S_c=tuple(range(0,n-j))       
        sdp.add_constraint((K*F[Elements_R[S_c]]-F[Elements_R[S]])== 0)
    
    ### A'_j>=0 ###
    
    for m in range(n+1):
        sdp.add_constraint(sum((-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*math.comb(n,j)*F[Elements_R[tuple(range(0,j))]] 
                               for j in range(m+1)) >= 0)
    
    ### K*B'_j-A'_j>=0 ###
    
    for m in range(n+1):   
                                    
        sdp.add_constraint(                              
        sum(
            (-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*(K*math.comb(n,n-j)*F[Elements_R[Swap_multiplication(tuple(range(0,j)),Generators)]]
                                                   -math.comb(n,j)*F[Elements_R[tuple(range(0,j))]])
            for j in range(m+1)) >= 0)
                                   
    
    ##### Adding Shadow constrains #####

    ### sum(K_j*A'_j)>=0 ###
    
    for j in range(n+1):
        sdp.add_constraint(sum(K_fun(n-j,i,n)*math.comb(n,i)*F[Elements_R[tuple(range(0,i))]] 
                               for i in range(n+1)) >= 0)
    
    """
    ##### Adding NPA constrains #####

    #sdp.add_constraint(F>>0)
    
    
    """
    for S in Elements:
        for T in Elements:

            i=len(S)
            j=len(T)
            t=len(tuple(set(S) & set(T)))
            
            l=i+j-2*t
            
            #sdp.add_constraint(F[Elements_R[S],Elements_R[T]]==
            #                   math.comb(n-l,t)*math.comb(l,j-t)*F[Elements_R[tuple(range(0,l))]])
            sdp.add_constraint(F[Elements_R[S],Elements_R[T]]==F[Elements_R[tuple(range(0,l))]])
                 
    """
    """
    for j in range(0,T):

        if not j in list2:

            sdp.add_constraint(F[j]==0)
            
    """
    list2=[]
    for i in range(0,n+1):
        list2.append(Elements_R[tuple(range(0,i))])
        
        
    for i in range(0,n+1):
        sdp.add_constraint(F[list2[i]]==Ap[i])
    
    
    for i in range(d):
        sdp.add_constraint((K*Ap[n-i]-Ap[i])== 0)
    
    #sdp.add_constraint(Ap[0]==K**4)
    #sdp.add_constraint(Ap[n]==K**3)
    
    ### A_j>=0 ###
    
    for m in range(n+1):
        #sdp.add_constraint(sum((-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*Ap[j] for j in range(m+1)) >= 0)
        A=0
        for j in range(m+1):
            A+=(-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*math.comb(n,j)*Ap[j]
        sdp.add_constraint(A>=0)            
            
    sol = sdp.solve(solver='mosek', primals=False)
    
    #print(np.round(np.array(F[:]),2))
    
    if sol.problemStatus == "feasible":
        return True                    
    elif sol.problemStatus == "infeasible":
        return False
    else:
        return sol.problemStatus 

    
D=2

Codes=[
[1,0,1],[1,1,1],
[2,0,2],[2,1,1],[2,2,1],
[3,0,2],[3,1,1],[3,2,1],[3,3,1],
[4,0,2],[4,1,2],[4,2,2],[4,3,1],[4,4,1],
[5,0,3],[5,1,3],[5,2,2],[5,3,1],[5,4,1],[5,5,1],
[6,0,4],[6,1,3],[6,2,2],[6,3,2],[6,4,2],[6,5,1],[6,6,1],
[7,0,3],[7,1,3],[7,2,2],[7,3,2],[7,4,2],[7,5,1],[7,6,1],[7,7,1],
]

#Codes=[[7,7,1]]
#Codes=[[6,6,1]]
Codes=[[10, 8, 2]]
#### Existing qubit codes ####

for code in Codes:
    
    n,K,d=code
    p=n
    check=SDP(n,2**K,d,p,D)
    
    if check != True:
        print("%s=%s" % (code,check))

print("Done")



#### Not existing qubit codes (distance+1) ####

for code in Codes:
    
    n,K,d=code
    p=n
    check=SDP(n,2**K,d+1,p,D)
    
    if check != False:
        print("[%s,%s,%s]=%s" % (code[0],code[1],code[2]+1,check))

print("Done")


[10, 8, 2]=False
Done
Done
